In [1]:
%pip show flask

Name: Flask
Version: 2.2.3
Summary: A simple framework for building complex web applications.
Home-page: https://palletsprojects.com/p/flask
Author: Armin Ronacher
Author-email: armin.ronacher@active-4.com
License: BSD-3-Clause
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: click, importlib-metadata, itsdangerous, Jinja2, Werkzeug
Required-by: dash, jupyter-dash
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip show flask-sqlalchemy

Name: Flask-SQLAlchemy
Version: 3.0.3
Summary: Add SQLAlchemy support to your Flask application.
Home-page: 
Author: 
Author-email: Armin Ronacher <armin.ronacher@active-4.com>
License: BSD-3-Clause
Location: /Users/mton/miniconda3/envs/torch-gpu/lib/python3.9/site-packages
Requires: Flask, SQLAlchemy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip show gunicorn

Note: you may need to restart the kernel to use updated packages.


#### -- [INIT ROUTES] --
***

In [2]:
from flask import Flask
from threading import Thread

# Init Flask app
app = Flask(__name__)

# Define a route
@app.route('/')
def hello_world():
    return "Hello, World!"

# Define Flask application with desired settings
def run_flask_app():
    app.run(port=5012, debug=True, use_reloader=False)

#### -- [START FLASK THREAD] --
***

In [3]:
flask_thread = Thread(target=run_flask_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5012
Press CTRL+C to quit
127.0.0.1 - - [15/Apr/2023 14:01:44] "GET / HTTP/1.1" 200 -


#### -- [EXIT FLASK THREAD] --
***

In [5]:
flask_thread.join(timeout=1)

In [1]:
# Check active ports
!lsof -i :5012

In [ ]:
!kill 22042